 # Anime Recommender System

In [1]:
#import library
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import random

In [11]:
anime = pd.read_csv ('dataanime.csv')
anime.head()

,Title,Type,Episodes,Status,Start airing,End airing,Starting season,Broadcast time,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,2009-4-5,2010-7-4,Spring,Sundays at 17:00 (JST),"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,Kimi no Na wa.,Movie,1,Finished Airing,2016-8-26,-,-,-,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,Gintama°,TV,51,Finished Airing,2015-4-8,2016-3-30,Spring,Wednesdays at 18:00 (JST),"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
3,Steins;Gate 0,TV,23,Currently Airing,2018-4-12,-,Spring,Thursdays at 01:35 (JST),Nitroplus,Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",23 min. per ep.,PG-13,9.16,12609,186331,1117,The dark untold story of Steins;Gate that lead...
4,Steins;Gate,TV,24,Finished Airing,2011-4-6,2011-9-14,Spring,Wednesdays at 02:05 (JST),"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...


In [12]:
print (f"Shape of anime dataset: {anime.shape}")

Shape of anime dataset: (1563, 20)


In [4]:
rating = pd.read_csv ('rating.csv')
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
print (f"Shape of rating dataset: {rating.shape}")

Shape of rating dataset: (7813737, 3)


In [13]:
anime.head()

,Title,Type,Episodes,Status,Start airing,End airing,Starting season,Broadcast time,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,2009-4-5,2010-7-4,Spring,Sundays at 17:00 (JST),"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,Kimi no Na wa.,Movie,1,Finished Airing,2016-8-26,-,-,-,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,Gintama°,TV,51,Finished Airing,2015-4-8,2016-3-30,Spring,Wednesdays at 18:00 (JST),"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
3,Steins;Gate 0,TV,23,Currently Airing,2018-4-12,-,Spring,Thursdays at 01:35 (JST),Nitroplus,Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",23 min. per ep.,PG-13,9.16,12609,186331,1117,The dark untold story of Steins;Gate that lead...
4,Steins;Gate,TV,24,Finished Airing,2011-4-6,2011-9-14,Spring,Wednesdays at 02:05 (JST),"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...


In [14]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1563 non-null   object 
 1   Type             1563 non-null   object 
 2   Episodes         1563 non-null   object 
 3   Status           1563 non-null   object 
 4   Start airing     1563 non-null   object 
 5   End airing       1563 non-null   object 
 6   Starting season  1563 non-null   object 
 7   Broadcast time   1563 non-null   object 
 8   Producers        1563 non-null   object 
 9   Licensors        1563 non-null   object 
 10  Studios          1563 non-null   object 
 11  Sources          1563 non-null   object 
 12  Genres           1563 non-null   object 
 13  Duration         1563 non-null   object 
 14  Rating           1563 non-null   object 
 15  Score            1563 non-null   float64
 16  Scored by        1563 non-null   int64  
 17  Members       

In [19]:
#mendapatkan index
indices = pd.Series(anime.index, index = anime['Title'])

In [20]:
# 2 Setup the TfidfVect and  CountVec
tfidf = TfidfVectorizer(stop_words = "english")
countvec = CountVectorizer(stop_words = "english")

In [21]:
# 3. Get Matrix for both (string based)
tfidf_mat = tfidf.fit_transform(anime['Description'])
countvect_mat = countvec.fit_transform(anime['Genres'])

In [22]:
# 4. Cosine Similarity Score
syn_similarity = linear_kernel(tfidf_mat, tfidf_mat)
genre_similarity = linear_kernel(countvect_mat, countvect_mat)

In [24]:
# Building Recommedation System
# Get Recommendation
def getRecommendation (title) :
    # Get the index of the anime
    idx = indices[title]
    
    # We have 2 Similarity Metrics
    ## 1.Synopsis Similarity
    ## 2.Genre Similarity
    
    score_1 = list(enumerate(syn_similarity[idx]))
    score_2 = list(enumerate(genre_similarity[idx]))
    
    #Sort the scores in reverse order
    
    score_1 = sorted(score_1, key = lambda x: x[0], reverse = False)
    score_2 = sorted(score_2, key = lambda x: x[0], reverse = False)
    
    #Average of the two Similarity (Cosine) Scores
    combined_score = [(idx, (sc_1 + sc_2) / 2) for (idx, sc_1), (_, sc_2) in zip(score_1, score_2)]
    
    #Sorting the Combined Score.
    combined_score = sorted(combined_score, key = lambda x: x[1], reverse = True)
    
    #Get ID of Top 10 Similar Animes
    anime_ids = [i[0] for i in combined_score[1:11]]
    
    #Returning the Top Anime Names.
    return anime ['Title'].iloc[anime_ids]
    

In [29]:
def showRecommendation(anime_name):
    g = getRecommendation(title = anime_name)
    
    print (f"Your Anime: {anime_name}\n\nRecommended Anime for you: \n")    
    for i, name in g.iteritems():
        print(name)

In [33]:
showRecommendation(random.choice(anime['Title']))

Your Anime: Chrno Crusade

Recommended Anime for you: 

InuYasha
InuYasha: Kanketsu-hen
Shounen Onmyouji
InuYasha: Kagami no Naka no Mugenjo
Kuroshitsuji
Basilisk: Kouga Ninpou Chou
Kuroshitsuji Movie: Book of the Atlantic
Otome Youkai Zakuro
InuYasha: Guren no Houraijima
InuYasha: Tenka Hadou no Ken
